In [2]:
import pandas as pd
import numpy as np
import py_stringmatching as sm
import py_stringsimjoin as ssj
from pandas_profiling import ProfileReport
import re
import nltk
from datetime import datetime
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from help_functions import *
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 1000)

In [30]:
dnb = pd.read_csv('../../datasets/authors/org/dnb.csv',sep=',')
dbp = pd.read_csv('../../datasets/authors/org/dbp.csv',sep=',')
viaf = pd.read_csv('../../datasets/authors/org/viaf.csv',sep=',')
wiki = pd.read_csv('../../datasets/authors/org/wiki.csv',sep=',')

In [31]:
dbp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31604 entries, 0 to 31603
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   URI        31604 non-null  object
 1   abstract   31604 non-null  object
 2   birthdate  27313 non-null  object
 3   deathdate  14137 non-null  object
 4   gender     31604 non-null  object
 5   label      31604 non-null  object
 6   links      31604 non-null  object
 7   work       7392 non-null   object
dtypes: object(8)
memory usage: 1.9+ MB


In [32]:
dnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9998 entries, 0 to 9997
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   uri           9998 non-null   object 
 1   birthdate     9449 non-null   object 
 2   deathdate     5765 non-null   object 
 3   forename      9772 non-null   object 
 4   gender        9862 non-null   object 
 5   nameAddition  72 non-null     object 
 6   personalName  226 non-null    object 
 7   label         9998 non-null   object 
 8   prefix        269 non-null    object 
 9   surname       9772 non-null   object 
 10  work          0 non-null      float64
dtypes: float64(1), object(10)
memory usage: 859.3+ KB


In [33]:
viaf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10186 entries, 0 to 10185
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   birthdate  10083 non-null  object
 1   deathdate  6079 non-null   object
 2   gender     9947 non-null   object
 3   label      10178 non-null  object
 4   work       7707 non-null   object
 5   uri        10186 non-null  object
dtypes: object(6)
memory usage: 477.6+ KB


In [34]:
wiki.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31480 entries, 0 to 31479
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   birthdate  28907 non-null  object
 1   deathdate  14654 non-null  object
 2   label      31480 non-null  object
 3   work       2566 non-null   object
 4   gender     31477 non-null  object
 5   uri        31480 non-null  object
dtypes: object(6)
memory usage: 1.4+ MB


#### DBPedia

In [35]:
dbp.rename(columns={'URI':'uri','label':'name'},inplace=True)

In [36]:
dbp.drop(columns=['abstract','links'],inplace=True)

In [52]:
dbp = dbp.applymap(lambda s: s.strip() if not pd.isnull(s) else s)

In [53]:
dbp.head()

,uri,birthdate,deathdate,gender,name,work
0,http://dbpedia.org/resource/Abbie_Hoffman,1936-11-30,1989-04-12,male,Abbie Hoffman,Steal_This_Book\tWoodstock_Nation_(book)
1,http://dbpedia.org/resource/Adam_Oehlenschläger,1779-11-14,1850-01-20,male,Adam Oehlenschläger,NaN
2,http://dbpedia.org/resource/Adam_Wiśniewski-Snerg,1937-01-01,1995-08-30,male,Adam Wiśniewski-Snerg,NaN
3,http://dbpedia.org/resource/Adrienne_Rich,1929-05-16,2012-03-27,female,Adrienne Rich,"On_Lies,_Secrets_and_Silence"
4,http://dbpedia.org/resource/Alessandro_Manzoni,1785-03-07,1873-05-22,male,Alessandro Manzoni,The_Betrothed_(Manzoni_novel)\tThe_Betrothed_(miniseries)


#### DNB

In [38]:
dnb.drop(columns=['forename','nameAddition','personalName','prefix','surname'],inplace=True)

In [39]:
dnb.rename(columns={'label':'name'},inplace=True)

In [41]:
dnb = dnb.applymap(lambda s: s.strip() if not pd.isnull(s) else s)

In [42]:
dnb.head()

,uri,birthdate,deathdate,gender,name,work
0,http://d-nb.info/gnd/119047438,1936-01-01,1989-01-01,notKnown,"Hoffman, Abbie",NaN
1,http://d-nb.info/gnd/119071606,1779-11-14,1850-01-20,male,"Oehlenschläger, Adam Gottlob",NaN
2,http://d-nb.info/gnd/1028840020,1937-01-01,1995-08-23,notKnown,"Wiśniewski-Snerg, Adam",NaN
3,http://d-nb.info/gnd/118744895,1929-05-16,2012-03-27,female,"Rich, Adrienne",NaN
4,http://d-nb.info/gnd/118577409,1785-03-07,1873-05-22,male,"Manzoni, Alessandro",NaN


#### WIKI

In [43]:
#wiki['uri'] = wiki['uri'].str.lower()

In [44]:
wiki['birthdate'] = wiki['birthdate'].str.lstrip('+').str.rstrip('Z').str.replace('T',' ')
wiki['deathdate'] = wiki['deathdate'].str.lstrip('+').str.rstrip('Z').str.replace('T',' ')

In [45]:
wiki.rename(columns={'label':'name'},inplace=True)

In [46]:
wiki = wiki.applymap(lambda s: s.strip() if not pd.isnull(s) else s)

In [47]:
wiki.head()

,birthdate,deathdate,name,work,gender,uri
0,1936-11-30 00:00:00,1989-04-12 00:00:00,Abbie Hoffman,Woodstock Nation\tSteal This Book,male,http://www.wikidata.org/entity/Q306514
1,1779-11-14 00:00:00,1850-01-20 00:00:00,Adam Oehlenschläger,The golden horns\tDer er et yndigt land\tAladdin,male,http://www.wikidata.org/entity/Q168356
2,1937-01-01 00:00:00,1995-08-30 00:00:00,Adam Wiśniewski-Snerg,NaN,male,http://www.wikidata.org/entity/Q351193
3,1929-05-16 00:00:00,2012-03-27 00:00:00,Adrienne Rich,"Diving into the Wreck: Poems 1971-1972\tOn Lies, Secrets and Silence\tThe Dream of a Common Lang...",female,http://www.wikidata.org/entity/Q270705
4,1785-03-07 00:00:00,1873-05-22 00:00:00,Alessandro Manzoni,The Betrothed\tIl Conte di Carmagnola\tAdelchi,male,http://www.wikidata.org/entity/Q1064


#### VIAF

In [48]:
#viaf['uri'] = viaf['uri'].str.lower()

In [49]:
viaf.rename(columns={'label':'name'},inplace=True)

In [50]:
viaf = viaf.applymap(lambda s: s.strip() if not pd.isnull(s) else s)

In [51]:
viaf.head()

,birthdate,deathdate,gender,name,work,uri
0,1936-11-30,1989-04-12,male,Abbie Hoffman,"Hoffman, Abbie.\t Steal this book\t Revolution for the hell of it",http://viaf.org/viaf/79408884
1,1779-11-14,1850-01-20,male,Adam Gottlob Oehlenschläger,Vaulundurs saga\t Correggio\t Nordens guder\t Hakon jarl\t Aladdin\t Sanct Hansaften-spil,http://viaf.org/viaf/27105324
2,1937-01-01,1995-08-30,male,Adam Wiśniewski-Snerg,"Robot\t Wiśniewski-Snerg, Adam\t Nagi cel",http://viaf.org/viaf/85523915
3,1929-05-16,2012-03-27,female,Adrienne Rich,"Rich, Adrienne Cecile (1929-2012).\t Rich, Adrienne\t Adrienne Rich with Diane Wood Middlebrook\...",http://viaf.org/viaf/16565
4,1785-03-07,1873-05-22,male,Alessandro Manzoni,"Storia della colonna infame\t Manzoni, Alessandro (1785-1873).\t Manzoni, Alessandro,\t Adelchi\...",http://viaf.org/viaf/14356


#### Densities

In [54]:
df = dnb
#(~(df.isnull()).sum() * 100) / len(df)
percent_missing = 100.0-(df.isnull().sum() * 100 / len(df))
densities_df = pd.DataFrame({'density': percent_missing})
densities_df.sort_values('density', ascending=False, inplace=True)
densities_df.drop(index=['uri']).head(10)

,density
name,100.000000
gender,98.639728
birthdate,94.508902
deathdate,57.661532
work,0.000000


In [55]:
df = viaf
#(~(df.isnull()).sum() * 100) / len(df)
percent_missing = 100.0-(df.isnull().sum() * 100 / len(df))
densities_df = pd.DataFrame({'density': percent_missing})
densities_df.sort_values('density', ascending=False, inplace=True)
densities_df.drop(index=['uri']).head(16)

,density
name,99.921461
birthdate,98.988808
gender,97.653642
work,75.662674
deathdate,59.679953


In [56]:
df = wiki
#(~(df.isnull()).sum() * 100) / len(df)
percent_missing = 100.0-(df.isnull().sum() * 100 / len(df))
densities_df = pd.DataFrame({'density': percent_missing})
densities_df.sort_values('density', ascending=False, inplace=True)
densities_df.drop(index=['uri']).head(16)

,density
name,100.000000
gender,99.990470
birthdate,91.826557
deathdate,46.550191
work,8.151207


In [57]:
df = dbp
#(~(df.isnull()).sum() * 100) / len(df)
percent_missing = 100.0-(df.isnull().sum() * 100 / len(df))
densities_df = pd.DataFrame({'density': percent_missing})
densities_df.sort_values('density', ascending=False, inplace=True)
densities_df.drop(index=['uri']).head(16)

,density
gender,100.000000
name,100.000000
birthdate,86.422605
deathdate,44.731680
work,23.389444


In [59]:
dbp.columns = ['_'.join(['dbp']+col.lower().split(' ')) for col in dbp.columns]
wiki.columns = ['_'.join(['wiki']+col.lower().split(' ')) for col in wiki.columns]
viaf.columns = ['_'.join(['viaf']+col.lower().split(' ')) for col in viaf.columns]
dnb.columns = ['_'.join(['dnb']+col.lower().split(' ')) for col in dnb.columns]

In [60]:
wiki.insert(0, 'wiki_id', range(0, wiki.shape[0]))
wiki['wiki_id'] = wiki['wiki_id'].apply(lambda x: 'wiki_'+str(x))
dnb.insert(0, 'dnb_id', range(0, dnb.shape[0]))
dnb['dnb_id'] = dnb['dnb_id'].apply(lambda x: 'dnb_'+str(x))
dbp.insert(0, 'dbp_id', range(0, dbp.shape[0]))
dbp['dbp_id'] = dbp['dbp_id'].apply(lambda x: 'dbp_'+str(x))
viaf.insert(0, 'viaf_id', range(0, viaf.shape[0]))
viaf['viaf_id'] = viaf['viaf_id'].apply(lambda x: 'viaf_'+str(x))

In [62]:
dbp.to_csv('../../datasets/authors/dbp.csv',index=False)
dnb.to_csv('../../datasets/authors/dnb.csv',index=False)
wiki.to_csv('../../datasets/authors/wiki.csv',index=False)
viaf.to_csv('../../datasets/authors/viaf.csv',index=False)

### Using the candidate sets from Anna

In [318]:
gs_dnb_train = pd.read_csv('../../datasets/authors/org/gs/DBPediaAuthors_DnbDataAuthors_gold_standard_train.csv',sep=';')
gs_dnb_test = pd.read_csv('../../datasets/authors/org/gs/DBPediaAuthors_DnbDataAuthors_gold_standard_test.csv',sep=';')

In [319]:
gs_viaf_train = pd.read_csv('../../datasets/authors/org/gs/DBPediaAuthors_VIAFDataAuthors_gold_standard_train.csv',sep=';')
gs_viaf_test = pd.read_csv('../../datasets/authors/org/gs/DBPediaAuthors_VIAFDataAuthors_gold_standard_test.csv',sep=';')

In [320]:
gs_wiki_train = pd.read_csv('../../datasets/authors/org/gs/DBPediaAuthors_WikiDataAuthors_gold_standard_train.csv',sep=';')
gs_wiki_test = pd.read_csv('../../datasets/authors/org/gs/DBPediaAuthors_WikiDataAuthors_gold_standard_test.csv',sep=';')

In [321]:
gs_dnb_train.head()

,source_id,target_id,matching
0,http://dbpedia.org/resource/justus_de_harduwijn,http://d-nb.info/gnd/172151457,True
1,http://dbpedia.org/resource/robert_heron_(writer),http://d-nb.info/gnd/100496962,True
2,http://dbpedia.org/resource/ariwara_no_narihira,http://d-nb.info/gnd/136763480,True
3,http://dbpedia.org/resource/nikolay_gnedich,http://d-nb.info/gnd/118695517,True
4,http://dbpedia.org/resource/j._meade_falkner,http://d-nb.info/gnd/119366266,True


In [322]:
dbp_dnb = pd.merge(dbp,gs_dnb_train,left_on='dbp_link_db',right_on='source_id')
dbp_dnb = pd.merge(dnb,dbp_dnb,left_on='dnb_uri',right_on='target_id')

In [329]:
dbp_dnb['label'] = dbp_dnb['matching'].apply(lambda x: 1 if x else 0)
dbp_dnb.drop(columns=['dnb_link_db','matching','source_id','target_id','dnb_blocking_key','dbp_blocking_key'],inplace=True)

In [330]:
dbp_dnb.head()

,dnb_id,dnb_uri,dnb_birthdate,dnb_deathdate,dnb_gender,dnb_name,dnb_work,dbp_id,dbp_uri,dbp_birthdate,dbp_deathdate,dbp_gender,dbp_name,dbp_work,dbp_link_db,label
0,dnb_3,http://d-nb.info/gnd/118744895,1929-05-16,2012-03-27,female,"Rich, Adrienne",NaN,dbp_3,http://dbpedia.org/resource/Adrienne_Rich,1929-05-16,2012-03-27,female,Adrienne Rich,"\tOn_Lies,_Secrets_and_Silence",http://dbpedia.org/resource/adrienne_rich,1
1,dnb_3,http://d-nb.info/gnd/118744895,1929-05-16,2012-03-27,female,"Rich, Adrienne",NaN,dbp_114,http://dbpedia.org/resource/Stan_Rice,1942-11-07,2002-12-09,male,Stan Rice,NaN,http://dbpedia.org/resource/stan_rice,0
2,dnb_4,http://d-nb.info/gnd/118577409,1785-03-07,1873-05-22,male,"Manzoni, Alessandro",NaN,dbp_4,http://dbpedia.org/resource/Alessandro_Manzoni,1785-03-07,1873-05-22,male,Alessandro Manzoni,\tThe_Betrothed_(Manzoni_novel)\tThe_Betrothed_(miniseries),http://dbpedia.org/resource/alessandro_manzoni,1
3,dnb_4,http://d-nb.info/gnd/118577409,1785-03-07,1873-05-22,male,"Manzoni, Alessandro",NaN,dbp_200,http://dbpedia.org/resource/Alejandro_Carrión,1915-03-11,1992-1-4,male,Alejandro Carrión,NaN,http://dbpedia.org/resource/alejandro_carrión,0
4,dnb_4,http://d-nb.info/gnd/118577409,1785-03-07,1873-05-22,male,"Manzoni, Alessandro",NaN,dbp_3524,http://dbpedia.org/resource/Jean_Baptiste_Lefebvre_de_Villebrune,1732-1-1,1809-10-7,male,Jean Baptiste Lefebvre de Villebrune,NaN,http://dbpedia.org/resource/jean_baptiste_lefebvre_de_villebrune,0


In [331]:
dbp_dnb.to_csv('../../candsets/authors/candset_dbp_dnb.csv',index=False)

In [332]:
dbp_viaf = pd.merge(dbp,gs_viaf_train,left_on='dbp_link_db',right_on='source_id')
dbp_viaf = pd.merge(viaf,dbp_viaf,left_on='viaf_uri',right_on='target_id')

In [333]:
dbp_viaf['label'] = dbp_viaf['matching'].apply(lambda x: 1 if x else 0)
dbp_viaf.drop(columns=['viaf_link_db','matching','source_id','target_id','viaf_blocking_key','dbp_blocking_key'],inplace=True)

In [334]:
dbp_viaf.head()

,viaf_id,viaf_birthdate,viaf_deathdate,viaf_gender,viaf_name,viaf_work,viaf_uri,dbp_id,dbp_uri,dbp_birthdate,dbp_deathdate,dbp_gender,dbp_name,dbp_work,dbp_link_db,label
0,viaf_3,1929-05-16,2012-03-27,female,Adrienne Rich,"Rich, Adrienne Cecile (1929-2012).\t Rich, Adrienne\t Adrienne Rich with Diane Wood Middlebrook\...",http://viaf.org/viaf/16565,dbp_3,http://dbpedia.org/resource/Adrienne_Rich,1929-05-16,2012-03-27,female,Adrienne Rich,"\tOn_Lies,_Secrets_and_Silence",http://dbpedia.org/resource/adrienne_rich,1
1,viaf_4,1785-03-07,1873-05-22,male,Alessandro Manzoni,"Storia della colonna infame\t Manzoni, Alessandro (1785-1873).\t Manzoni, Alessandro,\t Adelchi\...",http://viaf.org/viaf/14356,dbp_4,http://dbpedia.org/resource/Alessandro_Manzoni,1785-03-07,1873-05-22,male,Alessandro Manzoni,\tThe_Betrothed_(Manzoni_novel)\tThe_Betrothed_(miniseries),http://dbpedia.org/resource/alessandro_manzoni,1
2,viaf_4,1785-03-07,1873-05-22,male,Alessandro Manzoni,"Storia della colonna infame\t Manzoni, Alessandro (1785-1873).\t Manzoni, Alessandro,\t Adelchi\...",http://viaf.org/viaf/14356,dbp_200,http://dbpedia.org/resource/Alejandro_Carrión,1915-03-11,1992-1-4,male,Alejandro Carrión,NaN,http://dbpedia.org/resource/alejandro_carrión,0
3,viaf_4,1785-03-07,1873-05-22,male,Alessandro Manzoni,"Storia della colonna infame\t Manzoni, Alessandro (1785-1873).\t Manzoni, Alessandro,\t Adelchi\...",http://viaf.org/viaf/14356,dbp_254,http://dbpedia.org/resource/Gianna_Manzini,1896-03-24,1974-08-31,female,Gianna Manzini,NaN,http://dbpedia.org/resource/gianna_manzini,0
4,viaf_4,1785-03-07,1873-05-22,male,Alessandro Manzoni,"Storia della colonna infame\t Manzoni, Alessandro (1785-1873).\t Manzoni, Alessandro,\t Adelchi\...",http://viaf.org/viaf/14356,dbp_24740,http://dbpedia.org/resource/Bentley_Little,1960-1-1,NaN,male,Bentley Little,NaN,http://dbpedia.org/resource/bentley_little,0


In [335]:
dbp_viaf.shape

(15316, 16)

In [336]:
dbp_viaf.to_csv('../../candsets/authors/candset_dbp_viaf.csv',index=False)

In [337]:
dbp_wiki = pd.merge(dbp,gs_wiki_train,left_on='dbp_link_db',right_on='source_id')
dbp_wiki = pd.merge(wiki,dbp_wiki,left_on='wiki_uri',right_on='target_id')

In [338]:
dbp_wiki['label'] = dbp_wiki['matching'].apply(lambda x: 1 if x else 0)
dbp_wiki.drop(columns=['wiki_link_db','matching','source_id','target_id','wiki_blocking_key','dbp_blocking_key'],inplace=True)

In [339]:
dbp_wiki.head()

,wiki_id,wiki_birthdate,wiki_deathdate,wiki_name,wiki_work,wiki_gender,wiki_uri,dbp_id,dbp_uri,dbp_birthdate,dbp_deathdate,dbp_gender,dbp_name,dbp_work,dbp_link_db,label
0,wiki_3,1929-05-16 00:00:00,2012-03-27 00:00:00,Adrienne Rich,"Diving into the Wreck: Poems 1971-1972\tOn Lies, Secrets and Silence\tThe Dream of a Common Lang...",female,http://www.wikidata.org/entity/q270705,dbp_3,http://dbpedia.org/resource/Adrienne_Rich,1929-05-16,2012-03-27,female,Adrienne Rich,"\tOn_Lies,_Secrets_and_Silence",http://dbpedia.org/resource/adrienne_rich,1
1,wiki_5,1913-12-18 00:00:00,1987-09-30 00:00:00,Alfred Bester,The Demolished Man\tThe Stars My Destination,male,http://www.wikidata.org/entity/q286116,dbp_5,http://dbpedia.org/resource/Alfred_Bester,1913-12-18,1987-09-30,male,Alfred Bester,"\tThe_Stars_My_Destination\tStar_Light,_Star_Bright_(book)\tThe_Demolished_Man\tStarburst_(Alfre...",http://dbpedia.org/resource/alfred_bester,1
2,wiki_6,1944-02-09 00:00:00,NaN,Alice Walker,The Color Purple\tThe Third Life of Grange Copeland\tMeridian\tThe Temple of My Familiar\tPosses...,female,http://www.wikidata.org/entity/q215868,dbp_54,http://dbpedia.org/resource/Alice_Walker,1944-02-09,NaN,female,Alice Walker,\tThe_Color_Purple\tIn_Search_of_Our_Mothers'_Gardens\tMeridian_(novel)\tEveryday_Use\tThe_Third...,http://dbpedia.org/resource/alice_walker,1
3,wiki_7,1952-02-19 00:00:00,NaN,Amy Tan,The Joy Luck Club\tThe Bonesetter's Daughter,female,http://www.wikidata.org/entity/q234989,dbp_55,http://dbpedia.org/resource/Amy_Tan,1952-02-19,NaN,female,Amy Tan,\tThe_Joy_Luck_Club_(novel)\tThe_Bonesetter's_Daughter\tThe_Kitchen_God's_Wife\tSaving_Fish_from...,http://dbpedia.org/resource/amy_tan,1
4,wiki_8,1621-03-31 00:00:00,1678-08-16 00:00:00,Andrew Marvell,NaN,male,http://www.wikidata.org/entity/q365935,dbp_6,http://dbpedia.org/resource/Andrew_Marvell,1621-03-31,1678-08-16,male,Andrew Marvell,\tThe_Garden_(poem)\tTo_His_Coy_Mistress,http://dbpedia.org/resource/andrew_marvell,1


In [340]:
dbp_wiki.shape

(19228, 16)

In [341]:
dbp_wiki[dbp_wiki['label']==False]

,wiki_id,wiki_birthdate,wiki_deathdate,wiki_name,wiki_work,wiki_gender,wiki_uri,dbp_id,dbp_uri,dbp_birthdate,dbp_deathdate,dbp_gender,dbp_name,dbp_work,dbp_link_db,label
6,wiki_10,1896-02-01 00:00:00,1966-09-28 00:00:00,André Breton,NaN,male,http://www.wikidata.org/entity/q161955,dbp_9,http://dbpedia.org/resource/André_Malraux,1901-11-03,1976-11-23,male,André Malraux,\tMan's_Fate,http://dbpedia.org/resource/andré_malraux,0
7,wiki_11,1901-11-03 00:00:00,1976-11-23 00:00:00,André Malraux,Man's Fate,male,http://www.wikidata.org/entity/q151164,dbp_8,http://dbpedia.org/resource/André_Breton,1896-02-19,1966-09-28,male,André Breton,\tSurrealist_Manifesto\tLes_Champs_magnétiques\tNadja_(novel)\tManifestoes_of_Surrealism,http://dbpedia.org/resource/andré_breton,0
11,wiki_17,1931-03-16 00:00:00,2009-05-02 00:00:00,Augusto Boal,NaN,male,http://www.wikidata.org/entity/q434805,dbp_14,http://dbpedia.org/resource/August_Strindberg,1849-01-22,1912-05-14,male,August Strindberg,\tMiss_Julie\tThe_Ghost_Sonata\tThe_Red_Room_(Strindberg_novel)\tInferno_(Strindberg_novel)\tTo_...,http://dbpedia.org/resource/august_strindberg,0
14,wiki_18,1928-06-19 00:00:00,2002-03-31 00:00:00,Barry Took,NaN,male,http://www.wikidata.org/entity/q4864842,dbp_62,http://dbpedia.org/resource/Harry_Turtledove,1949-06-14,NaN,male,Harry Turtledove,\tThe_Two_Georges\tThe_Guns_of_the_South\tSouthern_Victory_Series\tCrosstime_Traffic\tWorldwar_S...,http://dbpedia.org/resource/harry_turtledove,0
16,wiki_19,1572-06-21 00:00:00,1637-08-06 00:00:00,Ben Jonson,NaN,male,http://www.wikidata.org/entity/q193857,dbp_100,http://dbpedia.org/resource/Samuel_Johnson,1709-09-18,1784-12-13,male,Samuel Johnson,"\tA_Dictionary_of_the_English_Language\tThe_History_of_Rasselas,_Prince_of_Abissinia",http://dbpedia.org/resource/samuel_johnson,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19222,wiki_31453,NaN,NaN,Richard Gerberding,NaN,male,http://www.wikidata.org/entity/q22097206,dbp_92,http://dbpedia.org/resource/Richard_Wright_(author),1908-09-04,1960-11-28,male,Richard Wright (author),\tNative_Son\tBlack_Boy\tUncle_Tom's_Children\tThe_Outsider_(Richard_Wright)\tNative_Son_(play)\...,http://dbpedia.org/resource/richard_wright_(author),0
19223,wiki_31457,1967-01-01 00:00:00,NaN,Saci Lloyd,NaN,female,http://www.wikidata.org/entity/q15731352,dbp_73,http://dbpedia.org/resource/Lloyd_Alexander,1924-01-30,2007-05-17,male,Lloyd Alexander,\tThe_Chronicles_of_Prydain\tWestmark_(novel)\tThe_Black_Cauldron_(novel)\tThe_Foundling_and_Oth...,http://dbpedia.org/resource/lloyd_alexander,0
19224,wiki_31457,1967-01-01 00:00:00,NaN,Saci Lloyd,NaN,female,http://www.wikidata.org/entity/q15731352,dbp_99,http://dbpedia.org/resource/Saki,1870-12-18,1916-11-13,male,Saki,\tThe_Westminster_Alice\tWhen_William_Came\tBeasts_and_Super-Beasts,http://dbpedia.org/resource/saki,0
19225,wiki_31458,NaN,NaN,Sarah Gerard,NaN,female,http://www.wikidata.org/entity/q20878388,dbp_41,http://dbpedia.org/resource/Gerald_of_Wales,1146-1-1,1223-1-1,male,Gerald of Wales,NaN,http://dbpedia.org/resource/gerald_of_wales,0


In [342]:
dbp_wiki.to_csv('../../candsets/authors/candset_dbp_wiki.csv',index=False)

#### Test Set

In [343]:
dbp_dnb_test = pd.merge(dbp,gs_dnb_test,left_on='dbp_link_db',right_on='source_id')
dbp_dnb_test = pd.merge(dnb,dbp_dnb_test,left_on='dnb_uri',right_on='target_id')

In [344]:
dbp_dnb_test['label'] = dbp_dnb_test['matching'].apply(lambda x: 1 if x else 0)
dbp_dnb_test.drop(columns=['dnb_link_db','matching','source_id','target_id','dnb_blocking_key','dbp_blocking_key'],inplace=True)

In [345]:
dbp_dnb_test.head()

,dnb_id,dnb_uri,dnb_birthdate,dnb_deathdate,dnb_gender,dnb_name,dnb_work,dbp_id,dbp_uri,dbp_birthdate,dbp_deathdate,dbp_gender,dbp_name,dbp_work,dbp_link_db,label
0,dnb_4,http://d-nb.info/gnd/118577409,1785-03-07,1873-05-22,male,"Manzoni, Alessandro",NaN,dbp_169,http://dbpedia.org/resource/Alessandro_Verri,1741-11-09,1816-09-23,male,Alessandro Verri,NaN,http://dbpedia.org/resource/alessandro_verri,0
1,dnb_5,http://d-nb.info/gnd/118658654,1913-01-01,1987-01-01,notKnown,"Bester, Alfred",NaN,dbp_147,http://dbpedia.org/resource/Wilfred_Owen,1893-03-18,1918-11-04,male,Wilfred Owen,NaN,http://dbpedia.org/resource/wilfred_owen,0
2,dnb_6,http://d-nb.info/gnd/118884433,1944-02-09,NaN,female,"Walker, Alice",NaN,dbp_144,http://dbpedia.org/resource/Walker_Percy,1916-05-28,1990-05-10,male,Walker Percy,\tThe_Last_Gentleman_(novel)\tThe_Moviegoer\tThe_Message_in_the_Bottle\tThe_Thanatos_Syndrome,http://dbpedia.org/resource/walker_percy,0
3,dnb_9,http://d-nb.info/gnd/122951786,1928-01-01,2012-01-01,notKnown,"Sarris, Andrew",NaN,dbp_6,http://dbpedia.org/resource/Andrew_Marvell,1621-03-31,1678-08-16,male,Andrew Marvell,\tThe_Garden_(poem)\tTo_His_Coy_Mistress,http://dbpedia.org/resource/andrew_marvell,0
4,dnb_10,http://d-nb.info/gnd/118515187,1896-02-18,1966-09-28,male,"Breton, André",NaN,dbp_8,http://dbpedia.org/resource/André_Breton,1896-02-19,1966-09-28,male,André Breton,\tSurrealist_Manifesto\tLes_Champs_magnétiques\tNadja_(novel)\tManifestoes_of_Surrealism,http://dbpedia.org/resource/andré_breton,1


In [346]:
dbp_dnb_test.to_csv('../../candsets/authors/candset_dbp_dnb_test.csv',index=False)

In [347]:
dbp_viaf_test = pd.merge(dbp,gs_viaf_test,left_on='dbp_link_db',right_on='source_id')
dbp_viaf_test = pd.merge(viaf,dbp_viaf_test,left_on='viaf_uri',right_on='target_id')

In [348]:
dbp_viaf_test['label'] = dbp_viaf_test['matching'].apply(lambda x: 1 if x else 0)
dbp_viaf_test.drop(columns=['viaf_link_db','matching','source_id','target_id','viaf_blocking_key','dbp_blocking_key'],inplace=True)

In [349]:
dbp_viaf_test.head()

,viaf_id,viaf_birthdate,viaf_deathdate,viaf_gender,viaf_name,viaf_work,viaf_uri,dbp_id,dbp_uri,dbp_birthdate,dbp_deathdate,dbp_gender,dbp_name,dbp_work,dbp_link_db,label
0,viaf_4,1785-03-07,1873-05-22,male,Alessandro Manzoni,"Storia della colonna infame\t Manzoni, Alessandro (1785-1873).\t Manzoni, Alessandro,\t Adelchi\...",http://viaf.org/viaf/14356,dbp_169,http://dbpedia.org/resource/Alessandro_Verri,1741-11-09,1816-09-23,male,Alessandro Verri,NaN,http://dbpedia.org/resource/alessandro_verri,0
1,viaf_5,1913-12-18,1987-09-30,male,Alfred Bester,"Demolished man\t Deceivers\t Golem 100\t Star light, star bright\t Computer connection\t Light f...",http://viaf.org/viaf/56605956,dbp_5,http://dbpedia.org/resource/Alfred_Bester,1913-12-18,1987-09-30,male,Alfred Bester,"\tThe_Stars_My_Destination\tStar_Light,_Star_Bright_(book)\tThe_Demolished_Man\tStarburst_(Alfre...",http://dbpedia.org/resource/alfred_bester,1
2,viaf_6,1621-03-31,1678-08-16,male,Andrew Marvell,"Upon Appleton House\t Marvell, Andrew,\t Prose works\t Poems.\t Works\t Character of popery\t Re...",http://viaf.org/viaf/39381003,dbp_6,http://dbpedia.org/resource/Andrew_Marvell,1621-03-31,1678-08-16,male,Andrew Marvell,\tThe_Garden_(poem)\tTo_His_Coy_Mistress,http://dbpedia.org/resource/andrew_marvell,1
3,viaf_6,1621-03-31,1678-08-16,male,Andrew Marvell,"Upon Appleton House\t Marvell, Andrew,\t Prose works\t Poems.\t Works\t Character of popery\t Re...",http://viaf.org/viaf/39381003,dbp_7,http://dbpedia.org/resource/Andrew_Sarris,1928-10-31,2012-06-20,male,Andrew Sarris,NaN,http://dbpedia.org/resource/andrew_sarris,0
4,viaf_7,1928-10-31,2012-06-20,male,Andrew Sarris,NaN,http://viaf.org/viaf/79132061,dbp_7,http://dbpedia.org/resource/Andrew_Sarris,1928-10-31,2012-06-20,male,Andrew Sarris,NaN,http://dbpedia.org/resource/andrew_sarris,1


In [351]:
dbp_viaf_test.shape

(3828, 16)

In [352]:
dbp_viaf_test.to_csv('../../candsets/authors/candset_dbp_viaf_test.csv',index=False)

In [353]:
dbp_wiki_test = pd.merge(dbp,gs_wiki_test,left_on='dbp_link_db',right_on='source_id')
dbp_wiki_test = pd.merge(wiki,dbp_wiki_test,left_on='wiki_uri',right_on='target_id')

In [354]:
dbp_wiki_test['label'] = dbp_wiki_test['matching'].apply(lambda x: 1 if x else 0)
dbp_wiki_test.drop(columns=['wiki_link_db','matching','source_id','target_id','wiki_blocking_key','dbp_blocking_key'],inplace=True)

In [355]:
dbp_wiki_test.head()

,wiki_id,wiki_birthdate,wiki_deathdate,wiki_name,wiki_work,wiki_gender,wiki_uri,dbp_id,dbp_uri,dbp_birthdate,dbp_deathdate,dbp_gender,dbp_name,dbp_work,dbp_link_db,label
0,wiki_4,1785-03-07 00:00:00,1873-05-22 00:00:00,Alessandro Manzoni,The Betrothed\tIl Conte di Carmagnola\tAdelchi,male,http://www.wikidata.org/entity/q1064,dbp_4,http://dbpedia.org/resource/Alessandro_Manzoni,1785-03-07,1873-05-22,male,Alessandro Manzoni,\tThe_Betrothed_(Manzoni_novel)\tThe_Betrothed_(miniseries),http://dbpedia.org/resource/alessandro_manzoni,1
1,wiki_8,1621-03-31 00:00:00,1678-08-16 00:00:00,Andrew Marvell,NaN,male,http://www.wikidata.org/entity/q365935,dbp_7,http://dbpedia.org/resource/Andrew_Sarris,1928-10-31,2012-06-20,male,Andrew Sarris,NaN,http://dbpedia.org/resource/andrew_sarris,0
2,wiki_9,1928-10-31 00:00:00,2012-06-20 00:00:00,Andrew Sarris,NaN,male,http://www.wikidata.org/entity/q507162,dbp_6,http://dbpedia.org/resource/Andrew_Marvell,1621-03-31,1678-08-16,male,Andrew Marvell,\tThe_Garden_(poem)\tTo_His_Coy_Mistress,http://dbpedia.org/resource/andrew_marvell,0
3,wiki_10,1896-02-01 00:00:00,1966-09-28 00:00:00,André Breton,NaN,male,http://www.wikidata.org/entity/q161955,dbp_8,http://dbpedia.org/resource/André_Breton,1896-02-19,1966-09-28,male,André Breton,\tSurrealist_Manifesto\tLes_Champs_magnétiques\tNadja_(novel)\tManifestoes_of_Surrealism,http://dbpedia.org/resource/andré_breton,1
4,wiki_13,1917-02-25 00:00:00,1993-11-22 00:00:00,Anthony Burgess,NaN,male,http://www.wikidata.org/entity/q217619,dbp_11,http://dbpedia.org/resource/Anthony_Burgess,1917-02-25,1993-11-22,male,Anthony Burgess,\tThe_Wanting_Seed\tTime_for_a_Tiger\tTremor_of_Intent:_An_Eschatological_Spy_Novel\tA_Vision_of...,http://dbpedia.org/resource/anthony_burgess,1


In [356]:
dbp_wiki_test.shape

(4807, 16)

In [357]:
dbp_wiki_test[dbp_wiki_test['label']==False]

,wiki_id,wiki_birthdate,wiki_deathdate,wiki_name,wiki_work,wiki_gender,wiki_uri,dbp_id,dbp_uri,dbp_birthdate,dbp_deathdate,dbp_gender,dbp_name,dbp_work,dbp_link_db,label
1,wiki_8,1621-03-31 00:00:00,1678-08-16 00:00:00,Andrew Marvell,NaN,male,http://www.wikidata.org/entity/q365935,dbp_7,http://dbpedia.org/resource/Andrew_Sarris,1928-10-31,2012-06-20,male,Andrew Sarris,NaN,http://dbpedia.org/resource/andrew_sarris,0
2,wiki_9,1928-10-31 00:00:00,2012-06-20 00:00:00,Andrew Sarris,NaN,male,http://www.wikidata.org/entity/q507162,dbp_6,http://dbpedia.org/resource/Andrew_Marvell,1621-03-31,1678-08-16,male,Andrew Marvell,\tThe_Garden_(poem)\tTo_His_Coy_Mistress,http://dbpedia.org/resource/andrew_marvell,0
7,wiki_16,1849-01-22 00:00:00,1912-05-14 00:00:00,August Strindberg,The Red Room\tThe Father\tMiss Julie\tInferno\tTo Damascus\tA Dream Play,male,http://www.wikidata.org/entity/q7724,dbp_15,http://dbpedia.org/resource/Augusto_Boal,1931-03-16,2009-05-02,male,Augusto Boal,NaN,http://dbpedia.org/resource/augusto_boal,0
10,wiki_25,1931-07-07 00:00:00,2009-06-02 00:00:00,David Eddings,The Belgariad,male,http://www.wikidata.org/entity/q354888,dbp_24,http://dbpedia.org/resource/David_Jones_(artist-poet),1895-11-01,1974-10-28,male,David Jones (artist-poet),\tIn_Parenthesis,http://dbpedia.org/resource/david_jones_(artist-poet),0
11,wiki_28,1952-11-28 00:00:00,NaN,David Zindell,NaN,male,http://www.wikidata.org/entity/q1177322,dbp_22,http://dbpedia.org/resource/David_Eddings,1931-07-07,2009-06-02,male,David Eddings,\tThe_Belgariad\tThe_Malloreon\tThe_Dreamers_(novel_series)\tThe_Elenium\tThe_Tamuli\tThe_Redemp...,http://dbpedia.org/resource/david_eddings,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4801,wiki_31439,NaN,NaN,Warren Motte,NaN,male,http://www.wikidata.org/entity/q26703084,dbp_9224,http://dbpedia.org/resource/Leo_Stein,1872-05-11,1947-07-29,male,Leo Stein,NaN,http://dbpedia.org/resource/leo_stein,0
4802,wiki_31445,1949-02-23 00:00:00,NaN,Noah Andre Trudeau,NaN,male,http://www.wikidata.org/entity/q21062211,dbp_8,http://dbpedia.org/resource/André_Breton,1896-02-19,1966-09-28,male,André Breton,\tSurrealist_Manifesto\tLes_Champs_magnétiques\tNadja_(novel)\tManifestoes_of_Surrealism,http://dbpedia.org/resource/andré_breton,0
4803,wiki_31454,NaN,NaN,Richard Hemming,NaN,male,http://www.wikidata.org/entity/q22278172,dbp_92,http://dbpedia.org/resource/Richard_Wright_(author),1908-09-04,1960-11-28,male,Richard Wright (author),\tNative_Son\tBlack_Boy\tUncle_Tom's_Children\tThe_Outsider_(Richard_Wright)\tNative_Son_(play)\...,http://dbpedia.org/resource/richard_wright_(author),0
4804,wiki_31467,NaN,NaN,Warren C. Brown,NaN,male,http://www.wikidata.org/entity/q22095385,dbp_37,http://dbpedia.org/resource/Fredric_Brown,1906-10-29,1972-03-11,male,Fredric Brown,\tThe_Fabulous_Clipjoint\tArena_(short_story)\tHoneymoon_in_Hell\tThe_Proofreaders'_Page_and_Oth...,http://dbpedia.org/resource/fredric_brown,0


In [358]:
dbp_wiki_test.to_csv('../../candsets/authors/candset_dbp_wiki_test.csv',index=False)